# Opis problema


Visedimenzionalni 0-1 problem ranca se klasifikuje kao NP tezak problem optimizacije. Moze se definisati kao:


**Ulaz**

* skup $O$ objekata {$o_1,...,o_n$}
* ranac $K$

**Izlaz**

* skup $O'$ objekata koji su u rancu

# Postavka problema

Visedimenzionalni 0-1 problem ranca je generalizacija 0-1 problema ranca. Sastoji se iz odabira podskupa objekata na taj nacin da je profit maximiziran pri cemu se postuju zadana ogranicenja.

$max\sum_{j=1}^{n}c_j\cdot x_j$

po:

$\sum_{j=1}^{n}a_{i,j}\cdot x_j\leq b_i,\forall i=1..m$

$x_j\epsilon\left \{ 0,1 \right \},1\leq j\leq n$

gdje je:

* $n$ je broj objekata
* $m$ je broj dimenzija ogranicenja ranca
* $c_j$ je profit objekta $j$ u rancu
* $x_j$ je binarna varijabla koja predstavlja da li se $j$-ti objekat stavlja ili ne stavlja u ranac
* $b_i$ je kapacitet $i$-te dimenzije
* $a_{i,j}$ je element matrice ogranicenja ranca, gdje je matrica dimenzije $nxm$

# Julia

Julia je dinamicki jezik visokog nivoa, visokih performansi za tehnicka racunanja sa sintaksom poznatom korisnicima drugih tehnickih kompjuterskih okruzenja (R, python). 

Ovaj rad je pisan u IJulia, kolaboracija Jupyter i Julia developera, namjenjen kao browser graficka sveska za Juliju.

# Testni primjer

Testni primjer preuzet iz [2]. Datoteka sadrzi: m, n, rjesenje, c, A, b i postavke za taboo pretragu koji se ignorisu.

In [1]:
dir = pwd()
cd("$dir\\input")

type parr
    n::Int
    m::Int
    rjesenje::Int
end

function openNth(br,sadrzaj)
    filein = open(sadrzaj[br])
    puni=""
    for line in eachline(filein)
        puni = puni * line
    end
    puni = strip(puni)
    #println(puni)
    close(filein)
    return puni
end

function parsiraj!(puni,c,A,b,par::parr)
    brojevi_str = split(puni,r" |\r|\n",keep=false)
    par.n = parse(Int,brojevi_str[1])
    par.m = parse(Int,brojevi_str[2])
    par.rjesenje = parse(Int,brojevi_str[3])
    
    cA = brojevi_str[4:(3+par.n)]
    tempA = brojevi_str[4+par.n:(3+par.n+par.n*par.m)]
    bA = brojevi_str[4+par.n+par.n*par.m:(3+par.n+par.n*par.m+par.m)]
    
    for i in bA
        ii = parse(Float64,i)
        push!(b,ii)
    end
    for i in cA
        ii = parse(Float64,i)
        push!(c,ii)
    end
    
    for poc in 1:par.n:(par.m*par.n)
        #println(poc)
        tt = []
        for i in 0:(par.n-1)
            #print(tempA[poc+i]*" ")
            push!(tt,parse(Float64,tempA[poc+i]))
        end

        push!(A,map(Float64,tt))
    end
end

parsiraj! (generic function with 1 method)

# GA
## Individua

In [2]:
type Individua
    Duzina
    Hromozom
    Fitnes::Float64
end
function Individua(Duzina::Int)
    Hromozom = BitArray(Duzina)
    rand!(Hromozom)
    return Individua(Duzina,Hromozom,0.0)
end
function Individua(Hromozom::BitArray)
    Duzina = length(Hromozom)
    return Individua(Duzina,Hromozom,0.0)
end

Individua

### Operacije nad individuom
Mutacija, nasumicno se bira lokus i negira vrijednost njegove allele.

In [3]:
function Mutiraj!(a::Individua)
    poz = rand(1:a.Duzina)
    a.Hromozom[poz] = !a.Hromozom[poz]
    return a
end

Mutiraj! (generic function with 1 method)

Ukrstanja

* _Ukrstanje_ - ukrstanje u dvije tacke, jedna funkcija vraca jedno djete po vjerovatnoci, druga vraca oboje
* _UkrstanjeN_ - ukrstanje u n tacka, gdje je n neparan broj, jedna funkcija vraca jedno djete po vjerovatnoci, druga vraca oboje
* _UkrstanjeUn_ - ukrstanje uniformno, allele se nasumicno biraju od roditelja

In [4]:
function Ukrstanje(a::Individua,b::Individua,p::Float64)
    tacka1 = rand(1:(a.Duzina-1))
    tacka2 = rand(1:(a.Duzina-1))
    if(tacka2<tacka1)
        temp = tacka2
        tacka2 = tacka1
        tacka1 = temp
    end
    #println(tacka)
    t1 = Individua([a.Hromozom[1:tacka1];b.Hromozom[(tacka1+1):tacka2]
        ;a.Hromozom[(tacka2+1):end]])
    t2 = Individua([b.Hromozom[1:tacka1];a.Hromozom[(tacka1+1):tacka2]
        ;b.Hromozom[(tacka2+1):end]])
    if(p>rand())
        return t1
    end
    return t2
end
function Ukrstanje(a::Individua,b::Individua)
    tacka1 = rand(1:(a.Duzina-1))
    tacka2 = rand(1:(a.Duzina-1))
    if(tacka2<tacka1)
        temp = tacka2
        tacka2 = tacka1
        tacka1 = temp
    end
    #println(tacka)
    t1 = Individua([a.Hromozom[1:tacka1];b.Hromozom[(tacka1+1):tacka2]
        ;a.Hromozom[(tacka2+1):end]])
    t2 = Individua([b.Hromozom[1:tacka1];a.Hromozom[(tacka1+1):tacka2]
        ;b.Hromozom[(tacka2+1):end]])

    return [t1;t2]
end
function UkrstanjeUn(a::Individua,b::Individua)
    rez1=BitArray(a.Duzina)
    rez2=BitArray(a.Duzina)
    
    for i in 1:a.Duzina
        p = rand()
        
        if(p<0.5)
            rez1[i]=a.Hromozom[i]
            rez2[i]=b.Hromozom[i]
        else
            rez2[i]=a.Hromozom[i]
            rez1[i]=b.Hromozom[i]
        end
    end
    
    return [Individua(rez1);Individua(rez2)]
end
function UkrstanjeN(a::Individua,b::Individua,p::Float64,n::Int)
    tacke = Array(Int,0)
    niz = 2:(a.Duzina-1)
    for s in 1:n
        t = rand(niz)
        push!(tacke,t)
    end
    sort!(tacke)

    kraj = length(tacke)
    h1 = [b.Hromozom[1:tacke[1]]]
    h2 = [a.Hromozom[1:tacke[1]]]
    for i in 1:2:(kraj-1)
        
        if((i+1)>kraj||(i+2)>kraj)
            break
        end
        temp = h1
        h1 = [temp;a.Hromozom[(tacke[i]+1):tacke[i+1]];
            b.Hromozom[(tacke[i+1]+1):tacke[i+2]]]
        h2 = [temp;b.Hromozom[(tacke[i]+1):tacke[i+1]];
            a.Hromozom[(tacke[i+1]+1):tacke[i+2]]]
    end
    
    h1 = [h1;b.Hromozom[(tacke[kraj]+1):end]]
    h2 = [h2;a.Hromozom[(tacke[kraj]+1):end]]
    
    t1 = Individua(h1)
    t2 = Individua(h2)    
    
    if(p>rand())
        return t1
    end
    return t2
end
function UkrstanjeN(a::Individua,b::Individua,n::Int)
    tacke = Array(Int,0)
    niz = 2:(a.Duzina-1)
    for s in 1:n
        t = rand(niz)
        push!(tacke,t)
    end
    sort!(tacke)

    kraj = length(tacke)
    h1 = [b.Hromozom[1:tacke[1]]]
    h2 = [a.Hromozom[1:tacke[1]]]
    for i in 1:2:(kraj-1)        
        if((i+1)>kraj||(i+2)>kraj)
            break
        end
        temp = h1
        h1 = [temp;a.Hromozom[(tacke[i]+1):tacke[i+1]];
            b.Hromozom[(tacke[i+1]+1):tacke[i+2]]]
        h2 = [temp;b.Hromozom[(tacke[i]+1):tacke[i+1]];
            a.Hromozom[(tacke[i+1]+1):tacke[i+2]]]
    end
    
    h1 = [h1;b.Hromozom[(tacke[kraj]+1):end]]
    h2 = [h2;a.Hromozom[(tacke[kraj]+1):end]]
    
    t1 = Individua(h1)
    t2 = Individua(h2)    
    
    return [t1;t2]
end

UkrstanjeN (generic function with 2 methods)

Evaluiraj! racuna fitnes Individue na osnovu unesenih A, c, b, n, m. Uvedena je kazna tezine izracunatog fitnes, za svako prekrseno ogranicenje.

In [5]:
function UFloatNiz(niz)
    rez=Array(Float64,0)
    for a in niz
        push!(rez,Float64(a))
    end
    return rez;
end
function Evauliraj!(a::Individua,A,c,b,n,m)
    tt=UFloatNiz(a.Hromozom)
    fitnes = tt'*c
    kazna = fitnes
    for i in 1:m
        sum = 0.0
        for j in 1:n
            sum += (A[i][j]*tt[j])
        end
        if sum<=b[i]
            #continue
        else
            fitnes-=kazna
        end
    end
    #println(fitnes)
    a.Fitnes = fitnes[1]
end

Evauliraj! (generic function with 1 method)

## Populacija

In [6]:
type Populacija
    DuzinaHromozoma::Int
    MaxGeneracija::Int
    VelicinaPopulacije::Int
    BrElite::Int
    BrMutacija::Int
    Iteracija::Int
    Populacija
end
function Populacija(Duzina::Int64)
    MaxGeneracija = 100*Duzina # 100 * brojvarijabli
    VelicinaPopulacije = 51 #neparn broj uvijek zbog neke logike
    BrElite = 1;
    BrMutacija = 15;
    Iteracija = 0;
    populacija = [];
    for a in 1:VelicinaPopulacije
        push!(populacija,Individua(Duzina))
    end
    return Populacija(Duzina,MaxGeneracija,VelicinaPopulacije,
    BrElite,BrMutacija,Iteracija,populacija)
end
function Populacija(Duzina::Int64,VelicinaPopulacije::Int64,
    BrMutacija::Int64)
    MaxGeneracija = 100*Duzina # 100 * brojvarijabli
    BrElite = 1;
    Iteracija = 0;
    populacija = [];
    for a in 1:VelicinaPopulacije
        push!(populacija,Individua(Duzina))
    end
    return Populacija(Duzina,MaxGeneracija,VelicinaPopulacije,
    BrElite,BrMutacija,Iteracija,populacija)
end

Populacija

### Evaluacija fitnesa svih jedinki

In [7]:
function EvalPop!(pop::Populacija,m,n,A,b,c)
    for a in pop.Populacija
        Evauliraj!(a,A,c,b,n,m)
    end
    sort!(pop.Populacija, by=x->(x.Fitnes), rev=true)
end
function EvalPop!(pop::Populacija)
    for a in pop.Populacija
        Evauliraj!(a)
    end
    sort!(pop.Populacija, by=x->(x.Fitnes), rev=true)
end

EvalPop! (generic function with 2 methods)

### Selekcija roditelja

* Dvije varijante turnira, jedna generise dva roditelja, druga generise m roditelja
* RWS - Roulette Wheel selekcija proporcionalno fitnesu

In [8]:
function Turnir(pop::Populacija,m::Int64,p::Float64)
    rez = [];
    for a in 1:m
        temp = [];
        t = rand(1:pop.VelicinaPopulacije)
        push!(temp,pop.Populacija[t])
        t = rand(1:pop.VelicinaPopulacije)
        push!(temp,pop.Populacija[t])
        sort!(temp,by=x->(x.Fitnes))
        if(p<rand())
            push!(rez,temp[1])
        else
            push!(rez,temp[2])
        end
    end
    return rez
end
function Turnir(pop::Populacija,p::Float64)
    return Turnir(pop,2,p);
end

Turnir (generic function with 2 methods)

In [9]:
function RWS(pop::Populacija)
    p = Array(Float64,pop.VelicinaPopulacije)
    suma = 0;
    for i in 1:pop.VelicinaPopulacije
        suma+=pop.Populacija[i].Fitnes
    end
    #fitnes moze biti negativan, zbog kazni
    max = 0.99
    for i in pop.VelicinaPopulacije:-1:1
        if(pop.Populacija[i].Fitnes<0)
            p[i] = max
            max-=0.01
        else
            p[i]=abs(pop.Populacija[i].Fitnes/suma)
        end
    end    
    j = 1;
    suma = p[j]
    u = rand()
    while (suma<u)
        j+=1
        suma+=(p[j])
    end
    return pop.Populacija[j]
end

RWS (generic function with 1 method)

### Ostale operacije

Funkcija DajElite, podrazumjeva sortiranu populaciju po fitnesu i vraca broj elita definisanih u objektu Populacija.

Funkcija Mutacija na osnovu broja definisanog u objektu Populacija mutira djecu. Po matlabovim specifikacijama, mutacija se sigurno desava, u objektu se definise kolko puta.

In [10]:
function DajElite(pop::Populacija)
    br = pop.BrElite
    return pop.Populacija[1:br]
end
function Mutacija(pop::Populacija,djeca)
    for i in 1:pop.BrMutacija
        br = rand(1:length(djeca))
        d = djeca[br]
        t = Mutiraj!(d)
        djeca[br] = t;
    end
end
function ProsjekFitnesa(pop::Populacija)
    suma = 0;
    for a in pop.Populacija 
        suma+=a.Fitnes 
    end
    suma/=length(pop.Populacija)
    return suma
end

ProsjekFitnesa (generic function with 1 method)

### Konstrukcija djece sa selekcijom i ukrstanjem

In [11]:
function DajDjecu(pop::Populacija,roditelji)
    rez = []
    brDjece = pop.VelicinaPopulacije - pop.BrElite
    while length(rez)<=brDjece
        r1 = roditelji[rand(1:length(roditelji))]
        r2 = roditelji[rand(1:length(roditelji))]
        d = Ukrstanje(r1,r2)
        push!(rez,d[1])
        push!(rez,d[2])
    end
    return rez
end
function DajDjecu(pop::Populacija)
    rez = []
    brDjece = pop.VelicinaPopulacije - pop.BrElite
    while length(rez)<=brDjece
        roditelji = Turnir(pop,0.9)
        r1 = roditelji[1]
        r2 = roditelji[2]
        d = Ukrstanje(r1,r2)
        push!(rez,d[1])
        push!(rez,d[2])
    end
    return rez
end

DajDjecu (generic function with 2 methods)

In [12]:
function DajDjecuUn(pop::Populacija)
    rez = []
    brDjece = pop.VelicinaPopulacije - pop.BrElite
    while length(rez)<=brDjece
        roditelji = Turnir(pop,0.9)
        r1 = roditelji[1]
        r2 = roditelji[2]
        d = UkrstanjeUn(r1,r2)
        push!(rez,d[1])
        push!(rez,d[2])
    end
    return rez
end

DajDjecuUn (generic function with 1 method)

In [13]:
function DajDjecuUnRWS(pop::Populacija)
    rez = []
    brDjece = pop.VelicinaPopulacije - pop.BrElite
    while length(rez)<=brDjece        
        r1 = RWS(pop)
        r2 = RWS(pop)
        d = UkrstanjeUn(r1,r2)
        push!(rez,d[1])
        push!(rez,d[2])
    end
    return rez
end

DajDjecuUnRWS (generic function with 1 method)

# Kombinacije izvrsavanja

## kombo1

Populacija sa 15 mutacija, velicina populacije 51. Konstruise se niz roditelja. Nasumicno odabrani roditelji konstruisu jedno dijete ukrstanjem u 2 tacke.

In [14]:
function kombo1(m,n,A,b,c,rjesenje)
    pop = Populacija(n)
    naj = 0
    for pop.Iteracija in 1:pop.MaxGeneracija
        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes

        elite = DajElite(pop)

        roditelji = 
        Turnir(pop,Int((pop.VelicinaPopulacije-pop.BrElite)/2),0.9)
        djeca = DajDjecu(pop,roditelji)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end

kombo1 (generic function with 1 method)

## kombo2

Populacija sa 15 mutacija, velicina populacije 51. Koristi turnir za odabir djece i ukrstanje u dvije tacke.

In [15]:
function kombo2(m,n,A,b,c,rjesenje)
    pop = Populacija(n)
    naj = 0
    for pop.Iteracija in 1:pop.MaxGeneracija        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes

        elite = DajElite(pop)
        djeca = DajDjecu(pop)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end

kombo2 (generic function with 1 method)

## kombo3

Populacija sa 15 mutacija, velicina populacije 51. Koristi turnir za odabir djece i uniformno ukrstanje.

In [16]:
function kombo3(m,n,A,b,c,rjesenje)
    pop = Populacija(n)
    naj=0
    for pop.Iteracija in 1:pop.MaxGeneracija        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes

        elite = DajElite(pop)
        djeca = DajDjecuUn(pop)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end

kombo3 (generic function with 1 method)

## kombo3_1

Populacija velicine 21, sa jednom mutacijom. Koristi turnir za odabir djece i uniformno ukrstanje.

In [17]:
function kombo3_1(m,n,A,b,c,rjesenje)
    pop = Populacija(n,21,1)
    naj=0
    for pop.Iteracija in 1:pop.MaxGeneracija        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes

        elite = DajElite(pop)
        djeca = DajDjecuUn(pop)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end

kombo3_1 (generic function with 1 method)

## kombo2_1

Populacija velicine 21, sa jednom mutacijom. Koristi turnir za odabir djece i ukrstanje u dvije tacke.

In [18]:
function kombo2_1(m,n,A,b,c,rjesenje)
    pop = Populacija(n,21,1)
    naj=0
    for pop.Iteracija in 1:pop.MaxGeneracija        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes

        elite = DajElite(pop)
        djeca = DajDjecu(pop)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end

kombo2_1 (generic function with 1 method)

## kombo3_2

Populacija velicine 51, sa jednom mutacijom. Koristi turnir za odabir djece i uniformno ukrstanje.

In [19]:
function kombo3_2(m,n,A,b,c,rjesenje)
    pop = Populacija(n,51,1)
    naj = 0
    for pop.Iteracija in 1:pop.MaxGeneracija        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes

        elite = DajElite(pop)
        djeca = DajDjecuUn(pop)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end

kombo3_2 (generic function with 1 method)

## kombo2_2

Populacija velicine 51, sa jednom mutacijom. Koristi turnir za odabir djece i ukrstanje u dvije tacke.

In [20]:
function kombo2_2(m,n,A,b,c,rjesenje)
    pop = Populacija(n,51,1)
    naj = 0
    for pop.Iteracija in 1:pop.MaxGeneracija        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes
        
        elite = DajElite(pop)
        djeca = DajDjecu(pop)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end

kombo2_2 (generic function with 1 method)

## kombo4
Populacija sa 15 mutacija, velicina populacije 51. Koristi RWS za odabir djece i uniformno ukrstanje.

In [21]:
function kombo4(m,n,A,b,c,rjesenje)
    pop = Populacija(n)
    naj = 0
    for pop.Iteracija in 1:pop.MaxGeneracija        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes
        
        elite = DajElite(pop)
        djeca = DajDjecuUnRWS(pop)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end

kombo4 (generic function with 1 method)

## kombo4_1
Populacija sa jednom mutacijom, velicina populacije 51. Koristi RWS za odabir djece i uniformno ukrstanje.

In [22]:
function kombo4_1(m,n,A,b,c,rjesenje)
    pop = Populacija(n,51,1)
    naj = 0
    for pop.Iteracija in 1:pop.MaxGeneracija        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes
        
        elite = DajElite(pop)
        djeca = DajDjecuUnRWS(pop)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end

kombo4_1 (generic function with 1 method)

# Rezultati
## Postavke za statistiku
Za statisticki pregled rezultata, napravljen je tip statistika.

In [23]:
type statistika
    algoritam
    vrijednosti
    vremena
    broj
    sumValue::Float64
    sumTime::Float64
    avgValue::Float64
    avgTime::Float64
end

Napravljen je konstruktor koji postavlja pocetne vrijednosti za statistiku.
Za manipulaciju statistike, funkcija dodaj! direktno modifikuje objekat, dodaje value i vrijeme u odgovarajuce nizove i racuna prosjeke.

In [24]:
function statistika(algoritam)
    vrijednosti = Array(Float64,0)
    vremena = Array(Float64,0)
    return statistika(algoritam,vrijednosti,vremena,0,0,0,0,0)
end
function dodaj!(sta::statistika,value,vrijeme)
    push!(sta.vrijednosti,value)
    push!(sta.vremena,vrijeme)
    sta.broj+=1
    sta.sumValue+=value
    sta.sumTime+=vrijeme
    sta.avgValue = sta.sumValue/sta.broj
    sta.avgTime = sta.sumTime/sta.broj
end
function ispis(niz)
    for sta in niz
        println("Algoritam $(sta.algoritam), pokrenut $(sta.broj) puta.")
        println("Prosjecno vrijeme izvrsavanje je $(sta.avgTime)s"*
        ", a prosjecno rjesenje je $(sta.avgValue)\n")
    end
end

ispis (generic function with 1 method)

## Puni program

Kupi testni primjer i 50 puta pozove sve kombinacije

In [25]:
sadrzaj = readdir()
stKombo1 = statistika("kombo1"); stKombo2 = statistika("kombo2")
stKombo2_1 = statistika("kombo2_1"); stKombo2_2 = statistika("kombo2_2")
stKombo3 = statistika("kombo3"); stKombo3_1 = statistika("kombo3_1")
stKombo3_2 = statistika("kombo3_2"); stKombo4 = statistika("kombo4")
stKombo4_1 = statistika("kombo4_1")
c = Array(Float64,0); b= Array(Float64,0);A = []; par = parr(0,0,0)
tic()
for file in 1:50
    puni = openNth(1,sadrzaj)

    c = Array(Float64,0); b= Array(Float64,0);A = []; par = parr(0,0,0)
    parsiraj!(puni,c,A,b,par)
    n = par.n; m= par.m; rjesenje = par.rjesenje

    tic()
    value = kombo1(m,n,A,b,c,rjesenje)
    dodaj!(stKombo1,value,toq())
    
    tic()
    value = kombo2(m,n,A,b,c,rjesenje)
    dodaj!(stKombo2,value,toq())
    
    tic()
    value = kombo3(m,n,A,b,c,rjesenje)
    dodaj!(stKombo3,value,toq())
    
    tic()
    value = kombo3_1(m,n,A,b,c,rjesenje)
    dodaj!(stKombo3_1,value,toq()) 
    
    tic()
    value = kombo2_1(m,n,A,b,c,rjesenje)
    dodaj!(stKombo2_1,value,toq())
    
    tic()
    value = kombo3_2(m,n,A,b,c,rjesenje)
    dodaj!(stKombo3_2,value,toq())
    
    tic()
    value = kombo2_2(m,n,A,b,c,rjesenje)
    dodaj!(stKombo2_2,value,toq())
    
    tic()
    value = kombo4(m,n,A,b,c,rjesenje)
    dodaj!(stKombo4,value,toq())
    
    tic()
    value = kombo4_1(m,n,A,b,c,rjesenje)
    dodaj!(stKombo4_1,value,toq())
    #break;

end
toc()

elapsed time: 719.268469093 seconds


719.268469093

In [27]:
println("n = $(par.n) broj objekata, dimenzije hromozoma\n"*
"m = $(par.m) broj dimenzija\nrjesenje = $(par.rjesenje)\n")
println("c = $c")
println("b = $b")
println("A = ")
for a in A
    println(a)
end

println()
ispis([stKombo1,stKombo2,stKombo2_1,stKombo2_2,stKombo3_1,
    stKombo3_2,stKombo4,stKombo4_1])

n = 15 broj objekata, dimenzije hromozoma
m = 10 broj dimenzija
rjesenje = 4015

c = [100.0,220.0,90.0,400.0,300.0,400.0,205.0,120.0,160.0,580.0,400.0,140.0,100.0,1300.0,650.0]
b = [550.0,700.0,130.0,240.0,280.0,310.0,110.0,205.0,260.0,275.0]
A = 
[8.0,24.0,13.0,80.0,70.0,80.0,45.0,15.0,28.0,90.0,130.0,32.0,20.0,120.0,40.0]
[8.0,44.0,13.0,100.0,100.0,90.0,75.0,25.0,28.0,120.0,130.0,32.0,40.0,160.0,40.0]
[3.0,6.0,4.0,20.0,20.0,30.0,8.0,3.0,12.0,14.0,40.0,6.0,3.0,20.0,5.0]
[5.0,9.0,6.0,40.0,30.0,40.0,16.0,5.0,18.0,24.0,60.0,16.0,11.0,30.0,25.0]
[5.0,11.0,7.0,50.0,40.0,40.0,19.0,7.0,18.0,29.0,70.0,21.0,17.0,30.0,25.0]
[5.0,11.0,7.0,55.0,40.0,40.0,21.0,9.0,18.0,29.0,70.0,21.0,17.0,35.0,25.0]
[0.0,0.0,1.0,10.0,4.0,10.0,0.0,6.0,0.0,6.0,32.0,3.0,0.0,70.0,10.0]
[3.0,4.0,5.0,20.0,14.0,20.0,6.0,12.0,10.0,18.0,42.0,9.0,12.0,100.0,20.0]
[3.0,6.0,9.0,30.0,29.0,20.0,12.0,12.0,10.0,30.0,42.0,18.0,18.0,110.0,20.0]
[3.0,8.0,9.0,35.0,29.0,20.0,16.0,15.0,10.0,30.0,42.0,20.0,18.0,120.0,20.0]

Algoritam ko

# Zakljucak

Kombinacije sa vise mutacije, tj sa vecom raznolikoscu, se bolje ponasaju. 

# Reference

1. http://tracer.lcc.uma.es/problems/multi01knap/knap.htm
1. http://www.cs.upc.edu/~mallba/public/library/TabuSearch/01mknp.1.html
1. http://uk.mathworks.com/help/gads/how-the-genetic-algorithm-works.html
1. http://www.mathworks.com/help/gads/some-genetic-algorithm-terminology.html
1. David E. Goldberg Genetic Algorithms in Search, Optimization, and Machine Learning
1. http://julialang.org/